In [1]:
import pandas as pd
from shapely import wkt
from sklearn.cluster import KMeans
import folium
from folium.plugins import Search, MarkerCluster, HeatMap

In [2]:
# todo | memasukkan csv 
df = pd.read_csv("D:/Semester 5 Teknik Geodesi/Pemograman Spasial/Propas Website/Restoran_Surakarta_Rating.csv", sep=';')

In [3]:
df['geometry'] = df['resto_geom'].apply(wkt.loads)
df['lon'] = df['geometry'].apply(lambda p: p.x)
df['lat'] = df['geometry'].apply(lambda p: p.y)

In [4]:
# todo | menambahkan dan memanggil algoritma K-Means
X = df[['lat', 'lon', 'rating']]
kmeans = KMeans(n_clusters=3, random_state=42)
df['cluster_id'] = kmeans.fit_predict(X)

c:\Users\HP\miniconda3\envs\python38\lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [5]:
#? Mendefinisikan popularitas
def kategori_rating(x):
    if x >= 4.3:
        return 'Ramai'
    elif x >= 3.5:
        return 'Sedang'
    else:
        return 'Normal'

df['popularity'] = df['rating'].apply(kategori_rating)
color_dict = {"Ramai": "red", "Sedang": "orange", "Normal": "blue"}

In [6]:
# ! Import BaseMap dari Folium
map_solo = folium.Map(location=[-7.56, 110.82], zoom_start=13)

folium.TileLayer("OpenStreetMap", name="Light Mode").add_to(map_solo)
folium.TileLayer("CartoDB Dark_Matter", name="Dark Mode").add_to(map_solo)

marker_cluster = MarkerCluster(name="Cluster Restoran").add_to(map_solo)


In [7]:

jenis_layers = {}
for jenis in df['jenis'].unique():
    jenis_layers[jenis] = folium.FeatureGroup(name=f"{jenis}", show=True)

# todo | Pembuatan marker setiap lokasi
for _, row in df.iterrows():
    google_maps_link = f"https://www.google.com/maps/dir/?api=1&destination={row['lat']},{row['lon']}"

    popup_html = f"""
    <div class='card p-2 shadow-sm' style='width:230px;font-family: Arial;'>
        <b style='font-size:15px;'>{row['nama_resto']}</b>
        ⭐ {row['rating']} | 🍽 {row['jenis']}
        <br><small class='text-muted'>🗣 {row['ulasan']} ulasan</small>
        <hr>
        <a class="btn btn-sm btn-primary w-100" href="{google_maps_link}" target="_blank" style="color:white;">
            📍 Buka Route Google Maps
        </a>
    </div>
    """

    marker = folium.Marker(
        [row['lat'], row['lon']],
        tooltip=row['nama_resto'],
        icon=folium.Icon(color=color_dict[row['popularity']])
    )
    marker.add_to(marker_cluster)
    marker.add_child(folium.Popup(popup_html, max_width=300))

    jenis_layers[row['jenis']].add_child(
        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=6,
            color=color_dict[row['popularity']],
            fill=True, fill_opacity=0.9
        )
    )

for layer in jenis_layers.values():
    layer.add_to(map_solo)

# todo | Search Bar
Search(
    layer=marker_cluster,
    search_label="nama_resto",
    placeholder="🔍 Cari restoran...",
    collapsed=False,
    position="topleft"
).add_to(map_solo)

# todo | Pembuatan script heatmap
HeatMap(df[['lat', 'lon', 'rating']].values.tolist(), name="Heatmap").add_to(map_solo)

# todo | pembuatan LEGENDA uuu sayy
legend = """
<div style='position:fixed; bottom:25px; left:20px; z-index:9999;
background:white;padding:10px;border-radius:8px;box-shadow:0 0 5px grey;'>
<b>Legenda Rating</b><br>
<i style='background:red;width:14px;height:14px;border-radius:50%;display:inline-block;'></i> Ramai (>4.3)<br>
<i style='background:orange;width:14px;height:14px;border-radius:50%;display:inline-block;'></i> Sedang (3.5–4.3)<br>
<i style='background:blue;width:14px;height:14px;border-radius:50%;display:inline-block;'></i> Normal (<3.5)
</div>
"""
map_solo.get_root().html.add_child(folium.Element(legend))

# todo | sidebar filter dan statistik
sidebar = f"""
<link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.1.0/dist/css/bootstrap.min.css">

<div id="sideMenu" class="bg-white shadow" 
style="position:fixed; top:0; right:-350px; width:350px; height:100%;
padding:20px; transition:0.4s; z-index:9999;">
<h5><b>📊 Statistik & Filter</b></h5><hr>

<p><b>Jumlah Restoran:</b> {len(df)}</p>
<p><b>Cluster K-Means:</b></p>
<ul>
    <li>Cluster 0 → {sum(df['cluster_id']==0)}</li>
    <li>Cluster 1 → {sum(df['cluster_id']==1)}</li>
    <li>Cluster 2 → {sum(df['cluster_id']==2)}</li>
</ul>

<hr>
<button class="btn btn-success w-100" onclick="openChart()">📈 Lihat Grafik Rating & Ulasan</button>
<hr>

<label><b>Filter Jenis Restoran</b></label>
<div class="accordion" id="accJenis">
"""

for j in sorted(df['jenis'].unique()):
    sidebar += f"<div><input type='checkbox' checked> {j}</div>"

sidebar += """
</div><br>
<button class="btn btn-danger w-100" onclick="toggleSidebar()">Tutup Sidebar</button>
</div>

<button onclick="toggleSidebar()" 
class="btn btn-primary rounded-circle shadow"
style="position:fixed; top:80px; right:20px; width:50px; height:50px; 
z-index:12000; border:none; font-size:20px;">
☰
</button>

<!-- Chart Modal -->
<div id="chartModal" style="display:none; position:fixed;top:10%;left:10%;
width:80%;height:70%;background:white;z-index:10000;border-radius:10px;box-shadow:0 0 8px black;">
<canvas id="chartResto"></canvas>
<button class="btn btn-danger w-100" onclick="closeChart()">Tutup Grafik</button>
</div>

<script src="https://cdn.jsdelivr.net/npm/chart.js"></script>

<script>
function toggleSidebar() {
  var sb = document.getElementById("sideMenu");
  sb.style.right = (sb.style.right == "0px") ? "-350px" : "0px";
}
function openChart() {
  document.getElementById("chartModal").style.display = "block";
}
function closeChart() {
  document.getElementById("chartModal").style.display = "none";
}

document.addEventListener("DOMContentLoaded", () => {
  new Chart(document.getElementById("chartResto"), {
    type: "bar",
    data: {
      labels: ['Rata2 Rating', 'Total Ulasan'],
      datasets: [{
        data: [""" + f"{df['rating'].mean():.2f}, {df['ulasan'].sum()}" +"""],
        backgroundColor: ['blue', 'green']
      }]
    }
  });
});
</script>
"""

In [ ]:
map_solo.get_root().html.add_child(folium.Element(sidebar))

# Layer Control
folium.LayerControl(collapsed=True).add_to(map_solo)

# Deployyy HTML nya broo
map_solo.save("MAP_Kuliner_Surakarta.html")